In [26]:
import numpy as np
import pandas as pd
import sqlite3 as sql
%run a_funciones.ipynb 
import openpyxl
from sklearn.preprocessing import MinMaxScaler
####Paquete para sistema basado en contenido ####
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors

In [27]:
def preprocesar():

    #### conectar_base_de_Datos#################
    conn=sql.connect('D:\\Octavo semestre\\Analitica 3\\Marketing\\proceso\\db_movies2') 
    cur=conn.cursor()
    

    ######## convertir datos crudos a bases filtradas por usuarios que tengan cierto número de calificaciones
    ejecutar_sql('D:\\Octavo semestre\\Analitica 3\\Marketing\\proceso\\preprocesamientomovies.sql', cur)

    ##### llevar datos que cambian constantemente a python ######
    movies=pd.read_sql('select * from movies3', conn )
    ratings=pd.read_sql('select * from ratings1', conn)
    usuarios=pd.read_sql('select distinct (userId) as user_id from ratings1',conn)


    ## eliminar filas que no se van a utilizar ###
    movies6=movies.drop(columns=['movieId','level_0','fecha','title','index'])

    return movies6, movies, conn, cur

In [10]:
##########################################################################
###############Función para entrenar modelo por cada usuario ##########
###############Basado en contenido todo lo visto por el usuario Knn#############################

In [28]:
def recomendar(user_id):
    
    movies6, movies, conn, cur= preprocesar()
    
    ratings2=pd.read_sql('select *from ratings1 where userId=:user',conn, params={'user':user_id})
    l_pelis_r=ratings2['movieId'].to_numpy()
    movies6[['movieId','title']]=movies[['movieId','title']]
    pelis_r=movies6[movies6['movieId'].isin(l_pelis_r)]
    pelis_r=pelis_r.drop(columns=['movieId','title'])
    pelis_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=pelis_r.groupby("indice").mean()
    
    
    pelis_nr=movies6[~movies6['movieId'].isin(l_pelis_r)]
    pelis_nr=pelis_nr.drop(columns=['movieId','title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(pelis_nr)
    dist, idlist = model.kneighbors(centroide)
    
    ids=idlist[0]
    recomend_b=movies.loc[ids][['title','movieId']]
    
 
    
    return recomend_b

In [29]:
recomendar(1)

,title,movieId
6338,Perfume: The Story of a Murderer (2006),48997
8365,Winter's Tale (2014),109191
3295,Africa: The Serengeti (1994),4458
3773,"Piano Teacher, The (La pianiste) (2001)",5269
4449,Camp (2003),6568
9370,Bridget Jones's Baby (2016),162590
9081,Hotel Transylvania 2 (2015),142997
8742,Eden (2014),127184
8639,Paradox (2010),119167
8258,Rush (2013),104913
